In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("paper")

import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame


from sqlalchemy import create_engine
engine = create_engine('postgresql://user:@localhost:5432/postgres')
conn = engine.connect() 
    
from geoalchemy2 import Geometry, WKTElement

%matplotlib inline


In [2]:
palette = {"A":"#5FCE72","B":"#0BC0ED","C":"#FFD419","D":"#FF4B19"}
D_highlight_palette = {"A":"#b2b2b2","B":"#b2b2b2","C":"#b2b2b2","D":"#FF4B19"}
D_highlight_palette2 = {"not_D":"#6b6b6b","D":"#FF4B19"}


# Purpose
- Aggregate back to 1930s boundaries, mostly for New York data
- Add in housing age data from 1950.


####  Weighted values function for 1930s boundaries.

In [3]:

features = {1930:['population','white','white_perc','colored','colored_perc','other','other_perc','median_homevalue','median_homevalue_adj','owned_perc'],
            1940:['population','white','white_perc','colored','colored_perc','other','other_perc','median_homevalue','median_homevalue_adj','homes','college','college_perc',
                  'owned_perc'],
            1950:['population','white','white_perc','colored','colored_perc','other','other_perc','median_homevalue','median_homevalue_adj','homes','college','college_perc',
 'median_income_adj','median_income','owned_perc'],
            1960:['population','white','white_perc','colored','colored_perc','other','other_perc','median_homevalue','median_homevalue_adj','homes','college','college_perc',
 'median_income_adj','median_income','hispanic_perc','hispanic','owned_perc'],
            1970:['population','white','white_perc','colored','colored_perc','other','other_perc','median_homevalue','median_homevalue_adj','homes','college','college_perc',
 'median_income_adj','median_income','hispanic_perc','hispanic','asian_perc','asian','unemployed','unemployed_perc','owned_perc'],
            1980:['population','white','white_perc','colored','colored_perc','other','other_perc','median_homevalue','median_homevalue_adj','homes','college','college_perc',
 'median_income_adj','median_income','hispanic_perc','hispanic','asian_perc','asian','unemployed','unemployed','unemployed_perc','owned_perc'],
            1990:['population','white','white_perc','colored','colored_perc','other','other_perc','median_homevalue','median_homevalue_adj','homes','college','college_perc',
 'median_income_adj','median_income','hispanic_perc','hispanic','asian_perc','asian','unemployed','unemployed','unemployed_perc','owned_perc'],
            2000:['population','white','white_perc','colored','colored_perc','other','other_perc','median_homevalue','median_homevalue_adj','homes','college','college_perc',
 'median_income_adj','median_income','hispanic_perc','hispanic','asian_perc','asian','unemployed','unemployed','unemployed_perc','owned_perc'],
            2010:['population','white','white_perc','colored','colored_perc','other','other_perc','median_homevalue','median_homevalue_adj','homes','college','college_perc',
 'median_income_adj','median_income','hispanic_perc','hispanic','asian_perc','asian','unemployed','asian_perc','asian','unemployed','unemployed','unemployed_perc','owned_perc'],
            2016:['population','white','white_perc','colored','colored_perc','other','other_perc','median_homevalue','median_homevalue_adj','homes','college','college_perc',
 'median_income_adj','median_income','hispanic_perc','hispanic','asian_perc','asian','unemployed','unemployed_perc','owned_perc']}

In [5]:
def get_weighted_values_1930_nyc(year1,year2, perc=0.01):
    "year1 and year2 are the same exit for 2016 and 2010 Census data"
    
    ## 2016 ACS data uses 2010 Census boundaries, though 2016 data not ultimately used in analysis. 
    
    sql ='''
    
    select * from (select x.*, y.gisjoin_1930, y.geom as geom_using,
                        st_area(st_intersection(x.geom,y.geom))/st_area(y.geom)::decimal as tract_1930_perc,
                        st_area(st_intersection(x.geom,y.geom))/st_area(x.geom)::decimal as tract_perc
                        from
                        (select distinct a.*,b.geom from (select distinct * from 
                            (select * from census_{} ) as a) as a, 
                            (select distinct * from 
                                (select * from us_tract_{}_conflated_4326) as b)  as b 
                            where a."GISJOIN" = b.gisjoin) as x, 
                        (select distinct gisjoin as gisjoin_1930, geom from (
                            select distinct * from us_tract_1930_conflated_4326) as y
                            ) as y
    where st_intersects(x.geom, y.geom)) as t where tract_perc >.01
    '''.format(year1,year2)
  
        
    ### Get all the census units who's area overlap is greater than the min threshold
    df_overlay_1 = gpd.GeoDataFrame.from_postgis(sql, conn,crs={'init':'epsg:4326'}, geom_col='geom_using' )
    print(df_overlay_1.shape)
    
    ### Create a df that groups by the 1930s tracts
    df_overlay=df_overlay_1.groupby(['gisjoin_1930']).count()[['index']]
    df_overlay=df_overlay.rename(columns={'index':'count'})
    
    df_overlay_2 = df_overlay_1[df_overlay_1['tract_1930_perc']>perc]
    for each in features[year1]: 

        ### for the other features, use a population-weighted average
        weighted_sum = df_overlay_2.groupby(['gisjoin_1930']).apply(lambda x: np.sum(x[each]*x['population']*x['tract_perc'])/ np.sum(x['population']*x['tract_perc']))
        df_overlay[each]=weighted_sum
        
        
    df_overlay=df_overlay.fillna(0)
    print(df_overlay.shape)
    return df_overlay.reset_index(),df_overlay_1.reset_index()

In [6]:
df_1930_ny = gpd.GeoDataFrame.from_postgis('''select distinct * from us_tract_1930_conflated_4326''', conn,crs={'init':'epsg:4326'}, geom_col='geom' )

def merge_overlay_geom_1930_ny(df,name):
    df_merge = df_1930_ny[['gisjoin','geom']].merge(df.reset_index(),
                   left_on='gisjoin',
                   right_on='gisjoin_1930')

    #### Create the population density meausure
    #### Update: Going to convert to geography and gonna do this in postgis 
    df_merge['population_density']=df_merge['population']/df_merge.geometry.to_crs({'init': 'epsg:3857'}).area
#     df_merge['geom'] = df_merge['geom'].apply(lambda x: WKTElement(x.wkt, srid=4326))
    df_merge.to_postgis('{}'.format(name), engine, if_exists='replace', index=False, 
                             dtype={'geom': Geometry('MultiPolygon', srid= 4326)})

/Users/wenfeixu/anaconda3/envs/gds/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [7]:
# for y in list(features.keys()):
#     print (y)
#     if y !=2016:
#         y1 = y
#         y2 = y   
#     else: 
#         y1 = y
#         y2= 2010
#     df,df_1 = get_weighted_values_1930_nyc(y1,y2)
#     print(df.shape)
    
#     merge_overlay_geom_1930_ny(df,'census_1930_overlay_{}'.format(y1))

## Housing Age

In [8]:
sql = f'''
    select * from "us_tract_1950_conflated_4326"
    '''
tracts_1950 = gpd.read_postgis(sql,con=conn,geom_col='geom')
tracts_1950 = tracts_1950.rename(columns={'gisjoin':'GISJOIN'})

In [10]:
housing_age = pd.read_csv('../data/NHGIS/census_1950/nhgis0105_ds82_1950_tract.csv')
housing_age = housing_age.rename(columns={
                                        'B0X001':'Total_units',
                                         'B0Y001':'1940_later',
                                         'B0Y002':'1930_1939',
                                         'B0Y003':'1920_1929',
                                         'B0Y004':'1910_earlier'})



In [11]:
tracts_1950_housing = tracts_1950.merge(housing_age[['GISJOIN','STATE','Total_units','1940_later','1930_1939','1920_1929','1910_earlier']],on='GISJOIN')
tracts_1950_housing['1940_later_perc'] = tracts_1950_housing['1940_later']/tracts_1950_housing['Total_units']
tracts_1950_housing['1930_1939_perc'] = tracts_1950_housing['1930_1939']/tracts_1950_housing['Total_units']
tracts_1950_housing['1920_1929_perc'] = tracts_1950_housing['1920_1929']/tracts_1950_housing['Total_units']
tracts_1950_housing['1910_earlier_perc'] = tracts_1950_housing['1910_earlier']/tracts_1950_housing['Total_units']
tracts_1950_housing = tracts_1950_housing.fillna(0)
tracts_1950_housing = tracts_1950_housing.fillna(0).replace([np.inf], 1)

In [44]:
### Run once
# tracts_1950_housing[['id','STATE', 'GISJOIN',  'Total_units','1940_later', '1930_1939', '1920_1929', '1910_earlier',
#        '1940_later_perc', '1930_1939_perc', '1920_1929_perc','1910_earlier_perc']]\
#         .rename(columns={'GISJOIN':'GISJOIN_housing'})\
#         .to_sql('census_1950_housing', engine, if_exists='replace', index=False)

In [133]:
### Run once
# tracts_1950_housing.to_file('scratch/housing_age_1950.shp')

In [12]:
area_gazeteer = pd.read_csv('../data/natl_census_to_holc_gazeteer.csv')

In [13]:
tracts_1950_housing = tracts_1950_housing.merge(area_gazeteer.groupby('state_name')[['region_name', 'division_name']].first().reset_index(),
                          left_on='STATE',right_on='state_name')

### Merge these back to 1940 (rest of the country) and 1930 (NYC) tract boundaries. 

Do this for 1930 and 1940 years

In [ ]:
year1 = 1950
year2 = 1950

features_housing  = features[year1]+['Total_units',
       '1940_later', '1930_1939', '1920_1929', '1910_earlier',
       '1940_later_perc', '1930_1939_perc', '1920_1929_perc',
       '1910_earlier_perc']
perc=0.01


sql_1930 = '''
select * from (select x.*, y.gisjoin_1930, y.geom as geom_using,
                    st_area(st_intersection(x.geom,y.geom))/st_area(y.geom)::decimal as tract_1930_perc,
                    st_area(st_intersection(x.geom,y.geom))/st_area(x.geom)::decimal as tract_perc
                    from
                    (select distinct a.*,b.geom from (select distinct * from 
                        (select a.*,b.* from 
                            census_1950 as a, 
                            census_1950_housing as b 
                            where a."GISJOIN" = b."GISJOIN_housing") as a) as a, 
                        (select distinct * from 
                            (select * from us_tract_1950_conflated_4326) as b)  as b 
                        where a."GISJOIN" = b.gisjoin) as x, 
                    (select distinct gisjoin as gisjoin_1930, geom from (
                        select distinct * from us_tract_1930_conflated_4326) as y
                        ) as y
where st_intersects(x.geom, y.geom)) as t where tract_perc >.01
'''

sql_1940 ='''
    
select * from (select x.*, y.gisjoin_1940, y.geom as geom_using,
                    st_area(st_intersection(x.geom,y.geom))/st_area(y.geom)::decimal as tract_1940_perc,
                    st_area(st_intersection(x.geom,y.geom))/st_area(x.geom)::decimal as tract_perc
                    from
                    (select distinct a.*,b.geom from (select distinct * from 
                        (select a.*,b.* from 
                            census_1950 as a, 
                            census_1950_housing as b 
                            where a."GISJOIN" = b."GISJOIN_housing") as a) as a, 
                        (select distinct * from 
                            (select * from us_tract_1950_conflated_4326) as b)  as b 
                        where a."GISJOIN" = b.gisjoin) as x, 
                    (select distinct gisjoin as gisjoin_1940, geom from (
                        select distinct * from us_tract_1940_conflated_4326) as y
                        ) as y
where st_intersects(x.geom, y.geom)) as t where tract_perc >.01
'''.format(year1,year2)


### Get all the census units who's area overlap is greater than the min threshold
df_overlay_1 = gpd.GeoDataFrame.from_postgis(sql_1940, conn,crs={'init':'epsg:4326'}, geom_col='geom_using' )
print(df_overlay_1.shape)

### Create a df that groups by the 1940s tracts
df_overlay=df_overlay_1.groupby(['gisjoin_1940']).count()[['index']]
df_overlay=df_overlay.rename(columns={'index':'count'})

df_overlay_2 = df_overlay_1[df_overlay_1['tract_1940_perc']>perc]
for each in features_housing: 

    ### for the other features, use a population-weighted average
    weighted_sum = df_overlay_2.groupby(['gisjoin_1940']).apply(lambda x: np.sum(x[each]*x['population']*x['tract_perc'])/ np.sum(x['population']*x['tract_perc']))
    df_overlay[each]=weighted_sum
df_overlay=df_overlay.fillna(0)




(9834, 41)


KeyboardInterrupt: 

In [55]:
df_merge.to_file('scratch/df_merge.geojson',driver='GeoJSON')

In [33]:
df_1940 = gpd.GeoDataFrame.from_postgis('''select distinct * from us_tract_1940_conflated_4326''', conn,crs={'init':'epsg:4326'}, geom_col='geom' )


In [38]:
# df_merge = df_overlay.copy()
# df_merge = df_1940[['gisjoin','geom']].merge(df_merge.reset_index(),
#                left_on='gisjoin',
#                right_on='gisjoin_1940')

# #### Create the population density meausure
# #### Update: Going to convert to geography and gonna do this in postgis 
# df_merge['population_density']=df_merge['population']/df_merge.geometry.to_crs({'init': 'epsg:3857'}).area
#     df_merge['geom'] = df_merge['geom'].apply(lambda x: WKTElement(x.wkt, srid=4326))
df_merge.to_postgis('{}'.format('census_1940_overlay_1950_new'), engine, if_exists='replace', index=False, 
                         dtype={'geom': Geometry('MultiPolygon', srid= 4326)})